# Lab 1 - Training a Machine Learning Model

In this lab you will setup the Azure Machine Learning service from code and create a classical machine learning model that logs metrics collected during model training.

## Download the datasets

The following cell will download the dataset used by this lab. Click into the following cell and use `Shift + Enter` to execute it

In [5]:
import uuid
import os

# Create a temporary folder to store locally relevant content for this notebook
tempFolderName = '/FileStore/aml_labs_{0}'.format(uuid.uuid4())
dbutils.fs.mkdirs(tempFolderName)
print('Content files will be saved to {0}'.format(tempFolderName))

filesToDownload = ['UsedCars_Clean.csv', 'UsedCars_Affordability.csv']

for fileToDownload in filesToDownload:
  downloadCommand = 'wget -O ''/dbfs{0}/{1}'' ''https://databricksdemostore.blob.core.windows.net/data/aml-labs/{1}'''.format(tempFolderName, fileToDownload)
  print(downloadCommand)
  os.system(downloadCommand)
  
#List all downloaded files
dbutils.fs.ls(tempFolderName)

Content files will be saved to /FileStore/aml_labs_e69f6adf-0308-46da-97d6-701eb891f0f5
wget -O /dbfs/FileStore/aml_labs_e69f6adf-0308-46da-97d6-701eb891f0f5/UsedCars_Clean.csv https://databricksdemostore.blob.core.windows.net/data/aml-labs/UsedCars_Clean.csv
wget -O /dbfs/FileStore/aml_labs_e69f6adf-0308-46da-97d6-701eb891f0f5/UsedCars_Affordability.csv https://databricksdemostore.blob.core.windows.net/data/aml-labs/UsedCars_Affordability.csv
 Out[ 1 ]: 
[FileInfo(path='dbfs:/FileStore/aml_labs_e69f6adf-0308-46da-97d6-701eb891f0f5/UsedCars_Affordability.csv', name='UsedCars_Affordability.csv', size=0),
 FileInfo(path='dbfs:/FileStore/aml_labs_e69f6adf-0308-46da-97d6-701eb891f0f5/UsedCars_Clean.csv', name='UsedCars_Clean.csv', size=0)]

## Train a simple model

The following cell loads the sampled dataset. Use `Shift + Enter` to execute the cell. Take a moment to look at the data loaded into the Pandas Dataframe - it contains data about used cars such as the price (in dollars), age (in years), KM (kilometers driven) and other attributes like weather it is automatic transimission, the number of doors, and the weight.

In [8]:
# Step 1 - load the data
########################
import numpy as np
import pandas as pd
import os

pathToCsvFile = os.path.join('/dbfs' + tempFolderName, 'UsedCars_Clean.csv')
df = pd.read_csv(pathToCsvFile, delimiter=',')
print(df)


Price Age KM FuelType HP MetColor Automatic CC Doors \
0 13500 23 46986 Diesel 90 1 0 2000 3 
1 13750 23 72937 Diesel 90 1 0 2000 3 
2 13950 24 41711 Diesel 90 1 0 2000 3 
3 14950 26 48000 Diesel 90 0 0 2000 3 
4 13750 30 38500 Diesel 90 0 0 2000 3 
5 12950 32 61000 Diesel 90 0 0 2000 3 
6 16900 27 94612 Diesel 90 1 0 2000 3 
7 18600 30 75889 Diesel 90 1 0 2000 3 
8 21500 27 19700 Petrol 192 0 0 1800 3 
9 12950 23 71138 Diesel 69 0 0 1900 3 
10 20950 25 31461 Petrol 192 0 0 1800 3 
11 19950 22 43610 Petrol 192 0 0 1800 3 
12 19600 25 32189 Petrol 192 0 0 1800 3 
13 21500 31 23000 Petrol 192 1 0 1800 3 
14 22500 32 34131 Petrol 192 1 0 1800 3 
15 22000 28 18739 Petrol 192 0 0 1800 3 
16 22750 30 34000 Petrol 192 1 0 1800 3 
17 17950 24 21716 Petrol 110 1 0 1600 3 
18 16750 24 25563 Petrol 110 0 0 1600 3 
19 16950 30 64359 Petrol 110 1 0 1600 3 
20 15950 30 67660 Petrol 110 1 0 1600 3 
21 16950 29 43905 Petrol 110 0 1 1600 3 
22 15950 28 56349 Petrol 110 1 0 1600 3 
23 16950 28 32220 Petrol 110 1 0 1600 3 
24 16250 29 25813 Petrol 110 1 0 1600 3 
25 15950 25 28450 Petrol 110 1 0 1600 3 
26 17495 27 34545 Petrol 110 1 0 1600 3 
27 15750 29 41415 Petrol 110 1 0 1600 3 
28 16950 28 44142 Petrol 110 0 0 1600 3 
29 17950 30 11090 Petrol 110 1 0 1600 3 
... ... ... ... ... ... ... ... ... ... 
1406 8950 70 44850 Petrol 110 1 0 1600 3 
1407 8250 69 44826 Petrol 110 0 0 1600 5 
1408 9250 80 44444 Petrol 110 1 0 1600 3 
1409 7900 75 43720 Petrol 110 1 0 1600 5 
1410 8500 78 43622 Petrol 86 1 0 1300 4 
1411 7950 76 43532 Petrol 110 0 0 1600 5 
1412 9950 69 42800 Petrol 110 1 0 1600 3 
1413 8750 74 42317 Petrol 107 1 1 1600 5 
1414 7500 80 42186 Petrol 110 1 0 1600 3 
1415 6950 72 42000 Petrol 110 1 0 1600 3 
1416 8950 79 40093 Petrol 110 0 0 1600 5 
1417 8750 79 39800 Petrol 107 0 1 1600 3 
1418 7750 73 39168 Petrol 86 0 0 1300 3 
1419 8450 75 38945 Petrol 110 1 0 1600 3 
1420 8150 76 36537 Petrol 110 0 1 1600 4 
1421 8500 78 36000 Petrol 86 0 1 1300 3 
1422 7600 78 36000 Petrol 110 1 0 1600 3 
1423 7950 80 35821 Petrol 86 0 1 1300 3 
1424 7750 73 34717 Petrol 86 0 0 1300 3 
1425 7950 80 34000 Petrol 86 1 0 1300 4 
1426 9950 78 30964 Petrol 110 0 1 1600 3 
1427 8950 71 29000 Petrol 86 1 1 1300 3 
1428 8450 72 26000 Petrol 86 0 0 1300 3 
1429 8950 78 24000 Petrol 86 1 1 1300 5 
1430 8450 80 23000 Petrol 86 0 0 1300 3 
1431 7500 69 20544 Petrol 86 1 0 1300 3 
1432 10845 72 19000 Petrol 86 0 0 1300 3 
1433 8500 71 17016 Petrol 86 0 0 1300 3 
1434 7250 70 16916 Petrol 86 1 0 1300 3 
1435 6950 76 1 Petrol 110 0 0 1600 5 

 Weight 
0 1165 
1 1165 
2 1165 
3 1165 
4 1170 
5 1170 
6 1245 
7 1245 
8 1185 
9 1105 
10 1185 
11 1185 
12 1185 
13 1185 
14 1185 
15 1185 
16 1185 
17 1105 
18 1065 
19 1105 
20 1105 
21 1170 
22 1120 
23 1120 
24 1120 
25 1120 
26 1120 
27 1120 
28 1120 
29 1120 
... ... 
1406 1050 
1407 1075 
1408 1050 
1409 1070 
1410 1000 
1411 1070 
1412 1050 
1413 1100 
1414 1050 
1415 1050 
1416 1114 
1417 1080 
1418 1015 
1419 1050 
1420 1075 
1421 1045 
1422 1050 
1423 1015 
1424 1015 
1425 1000 
1426 1080 
1427 1045 
1428 1015 
1429 1065 
1430 1015 
1431 1025 
1432 1015 
1433 1015 
1434 1015 
1435 1114 

[1436 rows x 10 columns]

We are going to try and build a model that can answer the question "Can I afford a car that is X months old and has Y kilometers on it, given I have $12,000 to spend?". We will engineer the label for affordable. Execute the following cell.

In [10]:
# Step 2 - add the affordable feature
######################################
df['Affordable'] = np.where(df['Price']<12000, 1, 0)
df_affordability = df[["Age","KM", "Affordable"]]
print(df_affordability)

Age KM Affordable
0 23 46986 0
1 23 72937 0
2 24 41711 0
3 26 48000 0
4 30 38500 0
5 32 61000 0
6 27 94612 0
7 30 75889 0
8 27 19700 0
9 23 71138 0
10 25 31461 0
11 22 43610 0
12 25 32189 0
13 31 23000 0
14 32 34131 0
15 28 18739 0
16 30 34000 0
17 24 21716 0
18 24 25563 0
19 30 64359 0
20 30 67660 0
21 29 43905 0
22 28 56349 0
23 28 32220 0
24 29 25813 0
25 25 28450 0
26 27 34545 0
27 29 41415 0
28 28 44142 0
29 30 11090 0
... ... ... ...
1406 70 44850 1
1407 69 44826 1
1408 80 44444 1
1409 75 43720 1
1410 78 43622 1
1411 76 43532 1
1412 69 42800 1
1413 74 42317 1
1414 80 42186 1
1415 72 42000 1
1416 79 40093 1
1417 79 39800 1
1418 73 39168 1
1419 75 38945 1
1420 76 36537 1
1421 78 36000 1
1422 78 36000 1
1423 80 35821 1
1424 73 34717 1
1425 80 34000 1
1426 78 30964 1
1427 71 29000 1
1428 72 26000 1
1429 78 24000 1
1430 80 23000 1
1431 69 20544 1
1432 72 19000 1
1433 71 17016 1
1434 70 16916 1
1435 76 1 1

[1436 rows x 3 columns]

We are going to train a Logistic Regression model in Azure Databricks. This type of model requires us to standardize the scale of our training features Age and KM, so we use the `StandardScaler` from Scikit-Learn to transform these features so that they have values centered with a mean around 0 (mostly between -2.96 and 1.29). Select Step 3 and execute the code. Observe the difference in min and max values between the un-scaled and scaled Dataframes. When we use Sci-Kit Learn, these models are trained on the driver node. Execute the following cell.

In [12]:
# Step 3 - Scale the numeric feature values
###########################################
X = df_affordability[["Age", "KM"]].values
y = df_affordability[["Affordable"]].values

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(pd.DataFrame(X).describe().round(2))
print(pd.DataFrame(X_scaled).describe().round(2))

/databricks/python/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
 warnings.warn(msg, _DataConversionWarning)
 0 1
count 1436.00 1436.00
mean 55.95 68533.26
std 18.60 37506.45
min 1.00 1.00
25% 44.00 43000.00
50% 61.00 63389.50
75% 70.00 87020.75
max 80.00 243000.00
 0 1
count 1436.00 1436.00
mean -0.00 0.00
std 1.00 1.00
min -2.96 -1.83
25% -0.64 -0.68
50% 0.27 -0.14
75% 0.76 0.49
max 1.29 4.65

Train the model by fitting a LogisticRegression against the scaled input features (X_scaled) and the labels (y). Execute the following cell.

In [14]:
# Step 4 - Fit a Logistic Regression
####################################
from sklearn import linear_model
# Create a linear model for Logistic Regression
clf = linear_model.LogisticRegression(C=1)

# we create an instance of Classifier and fit the data.
clf.fit(X_scaled, y)

/databricks/python/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 y = column_or_1d(y, warn=True)
 Out[ 9 ]: 
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
 intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
 penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
 verbose=0, warm_start=False)

Try prediction - if you set the age to 60 months and km to 40,000, does the model predict you can afford the car? Execute the cell and find out.

In [16]:
# Step 5 - Test the trained model's prediction
##############################################
age = 60
km = 40000

scaled_input = scaler.transform([[age, km]])
prediction = clf.predict(scaled_input)

print("Can I afford a car that is {} month(s) old with {} KM's on it?".format(age,km))
print("Yes (1)" if prediction[0] == 1 else "No (0)")

Can I afford a car that is 60 month(s) old with 40000 KM's on it?
Yes (1)

Now, let's get a sense for how accurate the model is. Execute the following cell. What was your model's accuracy?

In [18]:
# Step 6 - Measure the model's performance
##########################################
scaled_inputs = scaler.transform(X)
predictions = clf.predict(scaled_inputs)
print(predictions)

from sklearn.metrics import accuracy_score
score = accuracy_score(y, predictions)
print("Model Accuracy: {}".format(score.round(3)))

/databricks/python/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
 warnings.warn(msg, _DataConversionWarning)
[0 0 0 ... 1 1 1]
Model Accuracy: 0.926

One thing that can affect the model's performance is how much data of all the labeled training data available is used to train the model. In the next cell, you define a method that uses train_test_split from Scikit-Learn that will enable you to split the data using different percentages. Execute the cell to register this function.

In [20]:
# Step 7 - Define a method to experiment with different training subset sizes
#############################################################################
from sklearn.model_selection import train_test_split
full_X = df_affordability[["Age", "KM"]]
full_Y = df_affordability[["Affordable"]]

def train_eval_model(full_X, full_Y,training_set_percentage):
    train_X, test_X, train_Y, test_Y = train_test_split(full_X, full_Y, train_size=training_set_percentage, random_state=42)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(train_X)
    clf = linear_model.LogisticRegression(C=1)
    clf.fit(X_scaled, train_Y)

    scaled_inputs = scaler.transform(test_X)
    predictions = clf.predict(scaled_inputs)
    score = accuracy_score(test_Y, predictions)

    return (clf, score)

## Use Azure Machine Learning to log performance metrics
In the steps that follow, you will train multiple models using different sizes of training data and observe the impact on performance (accuracy). Each time you create a new model, you are executing a Run in the terminology of Azure Machine Learning service. In this case, you will create one Experiment and execute multiple Runs within it, each with different training percentages (and resultant varying accuracies).

Execute the following cell to quickly verify you have the Azure Machine Learning SDK installed on your cluster. If you get a version number back without error, you are ready to proceed.

In [23]:
# Step 8 - Verify AML SDK Installed
#####################################################################
import azureml.core
print("SDK Version:", azureml.core.VERSION)


SDK Version: 0.1.80

All Azure Machine Learning entities are organized within a Workspace. You can create an AML Workspace in the Azure Portal, but as the code in the following cell shows, you can also create a Workspace directly from code. Set the values for `subscription_id`, `resource_group`, `workspace_name` and `workspace_region` as directed by the comments. Execute Step 9. You will be prompted to log in to your Azure Subscription by the command output.

In [25]:
# Step 9 - Create a workspace
#####################################################################

#Provide the Subscription ID of your existing Azure subscription
subscription_id = "e223f1b3-d19b-4cfa-98e9-bc9be62717bc"

#Provide values for the Resource Group and Workspace that will be created
resource_group = "aml-workspace-z"
workspace_name = "aml-workspace-z"
workspace_region = 'westcentralus' # eastus, westcentralus, southeastasia, australiaeast, westeurope


import azureml.core

# import the Workspace class 
from azureml.core import Workspace

ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region,
    exist_ok = True)

print("Workspace Provisioning complete.")

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code B8Q99WZBG to authenticate.
Interactive authentication successfully completed.
Workspace Provisioning complete.

To begin capturing metrics, you must first create an Experiment and then call `start_logging()` on that Experiment. The return value of this call is a Run. This root run can have other child runs. When you are finished with an experiment run, use `complete()` to close out the root run. Execute the following cell to train four different models using differing amounts of training data and log the results to Azure Machine Learning.

In [27]:
# Step 10 - Create an experiment and log metrics for multiple training runs
###########################################################################
from azureml.core.run import Run
from azureml.core.experiment import Experiment

# start a training run by defining an experiment
myexperiment = Experiment(ws, "UsedCars_Experiment")
root_run = myexperiment.start_logging()

training_set_percentage = 0.25
run = root_run.child_run("Training_Set_Percentage-%0.5F" % training_set_percentage)
model, score = train_eval_model(full_X, full_Y, training_set_percentage)
print("With %0.2f percent of data, model accuracy reached %0.4f." % (training_set_percentage, score))
run.log("Training_Set_Percentage", training_set_percentage)
run.log("Accuracy", score)
run.complete()

training_set_percentage = 0.5
run = root_run.child_run("Training_Set_Percentage-%0.5F" % training_set_percentage)
model, score = train_eval_model(full_X, full_Y, training_set_percentage)
print("With %0.2f percent of data, model accuracy reached %0.4f." % (training_set_percentage, score))
run.log("Training_Set_Percentage", training_set_percentage)
run.log("Accuracy", score)
run.complete()

training_set_percentage = 0.75
run = root_run.child_run("Training_Set_Percentage-%0.5F" % training_set_percentage)
model, score = train_eval_model(full_X, full_Y, training_set_percentage)
print("With %0.2f percent of data, model accuracy reached %0.4f." % (training_set_percentage, score))
run.log("Training_Set_Percentage", training_set_percentage)
run.log("Accuracy", score)
run.complete()

training_set_percentage = 0.9
run = root_run.child_run("Training_Set_Percentage-%0.5F" % training_set_percentage)
model, score = train_eval_model(full_X, full_Y, training_set_percentage)
print("With %0.2f percent of data, model accuracy reached %0.4f." % (training_set_percentage, score))
run.log("Training_Set_Percentage", training_set_percentage)
run.log("Accuracy", score)
run.complete()

# Close out the experiment
root_run.complete()

/databricks/python/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 y = column_or_1d(y, warn=True)
With 0.25 percent of data, model accuracy reached 0.9192.
With 0.50 percent of data, model accuracy reached 0.9109.
With 0.75 percent of data, model accuracy reached 0.9081.
With 0.90 percent of data, model accuracy reached 0.9306.

Now that you have captured history for various runs, you can review the runs. You could use the Azure Portal for this - go to the Azure Portal, find your Azure Machine Learning Workspace, select Experiments and select the UsedCars_Experiment. However, in this case we will use the AML SDK to query for the runs. Execute the following cell to view the runs and their status.

In [29]:
# Step 11 - Review captured runs
################################
# Go to the Azure Portal, find your Azure Machine Learning Workspace, select Experiments and select the UsedCars_Experiment

# You can also query the run history using the SDK.
# The following command lists all of the runs for the experiment
runs = [r for r in root_run.get_children()]
print(runs)

[Run(Experiment: UsedCars_Experiment,
Id: 5127c916-a5e9-40e3-a5b2-085100c0ab03,
Type: None,
Status: Completed), Run(Experiment: UsedCars_Experiment,
Id: 9cf247f3-aaaa-4a8c-8fba-a2ed150571f8,
Type: None,
Status: Completed), Run(Experiment: UsedCars_Experiment,
Id: d1360130-204c-465c-bf58-70798882a71b,
Type: None,
Status: Completed), Run(Experiment: UsedCars_Experiment,
Id: c7492ea2-44b4-49d1-a458-e4a4ae2d4505,
Type: None,
Status: Completed)]